In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM 
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np

C:\Users\gstkn\anaconda3\envs\py4sci\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_parquet("small_tokenized_data.parquet")
IDS = df["input_ids"].tolist()
newIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    newIDS.append(each)

In [3]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [4]:
#PyTorch tensor with first 3 reviews from our training set and pass it to it
x = torch.tensor(newIDS[:8])
print(x.shape)
#y = torch.tensor()
#pooled = torch.tensor()
#this is the BERTs final layer output for each token
y, pooled = bert(x, output_all_encoded_layers=False)


# NEXT THING TO DO
#neural net model to go from pooled (vetors) to labels

C:\Users\gstkn\AppData\Local\Temp\ipykernel_5368\2978904793.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\utils\tensor_new.cpp:204.)
  x = torch.tensor(newIDS[:8])


torch.Size([8, 512])


In [5]:
print(pooled.shape)
print(y.shape)

torch.Size([8, 768])
torch.Size([8, 512, 768])


In [6]:
#get the star count AKA the y from half_data
df = pd.read_parquet("small_tokenized_data.parquet")
IDS = df["stars"].tolist()
starIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    starIDS.append(each)

In [7]:
import torch
# convert our data into a torch dataset
#need y data from the file
ratings = torch.tensor(starIDS[:8]).float()
data = torch.utils.data.TensorDataset(pooled, ratings)

# set up a data loader to get batches from our dataset
loader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle = True)

In [8]:
lin_layer_1 = torch.nn.Linear(768,800)
lin_layer_2 = torch.nn.Linear(800,1)

In [9]:
model = torch.nn.Sequential(   
    lin_layer_1,
    torch.nn.Sigmoid(),
    lin_layer_2,
    torch.nn.Sigmoid()
)

In [10]:
scoring_function = torch.nn.MSELoss()
num_epochs = 10

optimizer = torch.optim.SGD( model.parameters(), .001 )

In [11]:
def train():
    for step in range (num_epochs):

        for batch in loader:
            optimizer.zero_grad()
            batchX = batch[0]
            print(batchX.size())
            batchY = batch[1]
            print(batchY.size())
            # do a gradient descent update of out linear model.

            # put data through the model
            preds = model(batchX)
            #fixedPreds=preds[:,0]
            #print("preds: ", fixedPreds)
            # collect the error
            errors = scoring_function(preds, batchY)
            #print("errors: ", errors)

            # compute the gradient of the error with regard to the model paramenters
            errors.backward()

            # take a step in that direction
            optimizer.step()
            
            

In [12]:
train()

torch.Size([8, 768])
torch.Size([8])


C:\Users\gstkn\anaconda3\envs\py4sci\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([8, 768])
torch.Size([8])


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
print(ratings.size())
print(pooled.size())